<a href="https://colab.research.google.com/github/jvlax-y/jvlax-y/blob/main/Tugas_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub[pandas-datasets]

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prajitdatta/movielens-100k-dataset")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/movielens-100k-dataset


In [5]:
import os

# Cek isi folder hasil download
print("Files in dataset folder:")
print(os.listdir(path))

Files in dataset folder:
['ml-100k']


In [6]:
file_path = os.path.join(path, "ml-100k", "u.data")

In [7]:
df = pd.read_csv(file_path, sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode user_id dan movie_id ke angka mulai dari 0
user_enc = LabelEncoder()
df['user'] = user_enc.fit_transform(df['user_id'])

movie_enc = LabelEncoder()
df['movie'] = movie_enc.fit_transform(df['movie_id'])

# Total user dan movie unik
n_users = df['user'].nunique()
n_movies = df['movie'].nunique()

# Input: [user, movie], Output: rating
X = df[['user', 'movie']].values
y = df['rating'].values

# Split data train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense

# Input
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

# Embedding Layer (menyederhanakan representasi user & movie)
user_embedding = Embedding(input_dim=n_users, output_dim=15)(user_input)
movie_embedding = Embedding(input_dim=n_movies, output_dim=15)(movie_input)

# Flatten (ubah ke dimensi 1D)
user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

# Gabung user dan movie vector
concatenated = Concatenate()([user_vec, movie_vec])

# Hidden layers
dense = Dense(128, activation='relu')(concatenated)
output = Dense(1)(dense)  # output = rating prediksi

# Bangun model
model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 15)     │     14,145 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 15)     │     25,230 │ movie_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 15)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 15)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 30)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │      3,968 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        129 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 43,472 (169.81 KB)

 Trainable params: 43,472 (169.81 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit([X_train[:, 0], X_train[:, 1]], y_train,
          validation_data=([X_test[:, 0], X_test[:, 1]], y_test),
          epochs=10,
          batch_size=64)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 3.2302 - val_loss: 0.9148
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8873 - val_loss: 0.8920
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8683 - val_loss: 0.8828
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8559 - val_loss: 0.8871
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.8243 - val_loss: 0.8626
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.8133 - val_loss: 0.8628
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.7966 - val_loss: 0.8617
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.7683 - val_loss: 0.8543
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7556 - val_loss: 0.8514
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.7369 - val_loss: 0.8532


In [12]:
# Prediksi rating untuk data uji
y_pred = model.predict([X_test[:, 0], X_test[:, 1]])

# Bandingkan hasil
for i in range(10):
    print(f"User {X_test[i, 0]} | Movie {X_test[i, 1]} | Real: {y_test[i]} | Predicted: {round(y_pred[i][0], 2)}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
User 876 | Movie 380 | Real: 4 | Predicted: 3.5
User 814 | Movie 601 | Real: 3 | Predicted: 3.799999952316284
User 93 | Movie 430 | Real: 4 | Predicted: 3.2100000381469727
User 415 | Movie 874 | Real: 2 | Predicted: 2.859999895095825
User 499 | Movie 181 | Real: 2 | Predicted: 4.019999980926514
User 258 | Movie 1073 | Real: 3 | Predicted: 3.0899999141693115
User 597 | Movie 285 | Real: 5 | Predicted: 3.4200000762939453
User 885 | Movie 495 | Real: 4 | Predicted: 4.150000095367432
User 836 | Movie 14 | Real: 3 | Predicted: 3.4700000286102295
User 520 | Movie 183 | Real: 4 | Predicted: 3.049999952316284


In [15]:
user_id = 10
user_internal_id = user_enc.transform([user_id])[0]

# Ambil semua movie
all_movie_ids = df['movie'].unique()

# Ambil movie yang sudah ditonton user ini
watched_movies = df[df['user'] == user_internal_id]['movie'].values

# Cari movie yang belum ditonton
unwatched_movies = np.setdiff1d(all_movie_ids, watched_movies)

# Buat prediksi rating untuk semua film yang belum ditonton
user_input = np.full(len(unwatched_movies), user_internal_id)
pred_ratings = model.predict([user_input, unwatched_movies])

# Gabungkan dan ambil top 5
recommendations = list(zip(unwatched_movies, pred_ratings.flatten()))
recommendations.sort(key=lambda x: x[1], reverse=True)
top_5 = recommendations[:5]

print("\n🎬 Top 5 Rekomendasi untuk User 10:")
for movie_id, pred_rating in top_5:
    original_movie_id = movie_enc.inverse_transform([movie_id])[0]
    print(f"Movie ID: {original_movie_id} | Predicted Rating: {round(pred_rating, 2)}")

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

🎬 Top 5 Rekomendasi untuk User 10:
Movie ID: 1327 | Predicted Rating: 0.05000000074505806
Movie ID: 1379 | Predicted Rating: 0.05000000074505806
Movie ID: 103 | Predicted Rating: 0.05000000074505806
Movie ID: 1541 | Predicted Rating: 0.05000000074505806
Movie ID: 131 | Predicted Rating: 0.05000000074505806


In [16]:
# Path ke file u.item (ganti kalau lokasinya beda)
item_path = os.path.join(path, "ml-100k", "u.item")

# Load data item (pakai encoding='latin-1' biar karakter aman)
item_df = pd.read_csv(item_path, sep='|', encoding='latin-1', header=None,
                      names=['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)])

# Kita hanya butuh kolom id & title
movie_titles = item_df[['movie_id', 'title']]

In [18]:
# Top 5 rekomendasi (lanjutan dari sebelumnya)
print("\nTop 5 Rekomendasi untuk User 10:")
for movie_id, pred_rating in top_5:
    original_movie_id = movie_enc.inverse_transform([movie_id])[0]

    # Ambil judul film berdasarkan movie_id asli
    title = movie_titles[movie_titles['movie_id'] == original_movie_id]['title'].values
    title = title[0] if len(title) > 0 else f"Movie ID {original_movie_id}"

    print(f"{title} — Predicted Rating: {round(pred_rating, 2)}")


Top 5 Rekomendasi untuk User 10:
Captives (1994) — Predicted Rating: 0.05000000074505806
Love and Other Catastrophes (1996) — Predicted Rating: 0.05000000074505806
All Dogs Go to Heaven 2 (1996) — Predicted Rating: 0.05000000074505806
Beans of Egypt, Maine, The (1994) — Predicted Rating: 0.05000000074505806
Breakfast at Tiffany's (1961) — Predicted Rating: 0.05000000074505806
